# Pumped-Storage Optimisation with Genetic Algorithm and MILP

In [28]:
import pandas as pd
import numpy as np
import plotnine as pn
import plotly.graph_objs as go
import plotly.express as px
from tqdm.notebook import tqdm
from IPython.display import clear_output, display
import os
from itertools import product
import datetime

# Import own implementations
from milp import MILP
import genetic
from genetic import GA_Actions_Elite, GA_Actions_Tournament

background_colour = "#F2F2F2"
pn.theme_set(
    pn.theme_classic()
    + pn.theme(
        text=pn.element_text(family="monospace"),
        plot_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        panel_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        legend_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
    )
)

%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


## Reading the Price data

In [29]:
df = pd.read_csv("../01 - Data/example_week.csv")
df.head(2)

,spot,utc_time
0,101.54,2022-01-01 00:00:00+00:00
1,52.13,2022-01-01 01:00:00+00:00


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   spot      168 non-null    float64
 1   utc_time  168 non-null    object 
dtypes: float64(1), object(1)
memory usage: 2.8+ KB


## The Power Plant

In [31]:
plant_params = {
    "EFFICIENCY": 0.75,
    "MAX_STORAGE_M3": 5000,
    "MIN_STORAGE_M3": 0,
    "TURBINE_POWER_MW": 100,
    "PUMP_POWER_MW": 100,
    "TURBINE_RATE_M3H": 500,
    "MIN_STORAGE_MWH": 0,
    "INITIAL_WATER_LEVEL_PCT": 0,
}
plant_params["INITIAL_WATER_LEVEL"] = (
    plant_params["INITIAL_WATER_LEVEL_PCT"] * plant_params["MAX_STORAGE_M3"]
)
plant_params["PUMP_RATE_M3H"] = (
    plant_params["TURBINE_RATE_M3H"] * plant_params["EFFICIENCY"]
)
plant_params["MAX_STORAGE_MWH"] = (
    plant_params["MAX_STORAGE_M3"] / plant_params["TURBINE_RATE_M3H"]
) * plant_params["TURBINE_POWER_MW"]

## MILP

In [32]:
milp_solver = MILP(plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"])

In [33]:
milp_model, milp_status, milp_profile = milp_solver.solve()

In [34]:
milp_status

'Optimal'

In [35]:
milp_profile.head()

,water_level,action,colour_id,utc_time,spot
0,0.0,0,nothing,2022-01-01 00:00:00+00:00,101.54
1,375.0,-1,pump,2022-01-01 01:00:00+00:00,52.13
2,750.0,-1,pump,2022-01-01 02:00:00+00:00,20.78
3,1125.0,-1,pump,2022-01-01 03:00:00+00:00,15.66
4,1500.0,-1,pump,2022-01-01 04:00:00+00:00,21.47


## Training

### Elite Selection

In [38]:
best_config = {
    "MUTPB": 1,
    "POP_SIZE": 200,
    "INITIAL_MUTATION_RATE": 0.05,
    "FINAL_MUTATION_RATE": 0.01,
    "INITIAL_EXPLORATION": 0.66,
    "ELITISM": 0.1,
}

In [37]:
ga_solver = GA_Actions_Elite(
    plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"]
)

population, fitnesses, best_ind, history = ga_solver.train(
    config=best_config,
    total_generations=500,
    tune_mode=False,
)

c:\Users\mathi\miniconda3\envs\general\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
c:\Users\mathi\miniconda3\envs\general\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.


  0%|          | 0/500 [00:00<?, ?it/s]

In [39]:
history

,generation,best,average,worst
0,0,-357517.0,-1745003.685,-4262540.0
1,1,-198758.0,-1024268.905,-2393843.0
2,2,-124727.0,-802473.155,-2550269.0
3,3,-61664.0,-671469.870,-1835851.0
4,4,-69575.0,-592984.515,-1538219.0
...,...,...,...,...
495,495,267044.0,-9535.510,-554587.0
496,496,264102.0,-9626.360,-615277.0
497,497,264102.0,-2290.885,-515947.0
498,498,272384.0,-8946.390,-721746.0


In [ ]:
history.to_csv(f"./Training Results/{datetime.datetime.now().strftime('%Y%m%d%H%M')}training_history_elite.csv")

In [41]:
best_ind

[-1,
 -1,
 -1,
 -1,
 -1,
 -1,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 -1,
 0,
 -1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 0,
 0,
 0,
 0,
 -1,
 -1,
 -1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 0,
 -1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 -1,
 0,
 -1,
 -1,
 -1,
 -1,
 -1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 -1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 -1,
 0,
 0,
 0,
 -1,
 -1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1,
 0,
 -1,
 -1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 -1,
 1,
 0]

### Tournament Selection

Take the results from tuning

In [ ]:
ga_solver = GA_Actions_Tournament(
    plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"]
)

population, fitnesses, best_ind, history = ga_solver.train(
    config={
        "MUTPB": 1.0,
        "POP_SIZE": 100,
        "CXPB": 0.3,
        "INITIAL_MUTATION_RATE": 0.05,
        "FINAL_MUTATION_RATE": 0.005,
        "TOURNAMENT_SIZE": 4,
    },
    total_generations=500,
    tune_mode=False,
)

  0%|          | 0/500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
history

,generation,best,average,worst
0,0,-293032.0,-9980398.028,-10460514.0
1,1,-297433.0,-9738494.206,-10389603.0
2,2,-243306.0,-9532594.974,-10409811.0
3,3,-161953.0,-9400322.812,-10345659.0
4,4,-153819.0,-9079379.224,-10440012.0
...,...,...,...,...
295,295,298162.0,-442471.250,-9750296.0
296,296,298162.0,-142067.478,-9744398.0
297,297,298162.0,-202000.952,-9749372.0
298,298,298162.0,-82016.506,-9722400.0


In [ ]:
history.to_csv(f"./Training Results/{datetime.datetime.now().strftime('%Y%m%d%H%M')}training_history_tournament.csv")

In [ ]:
(
    pn.ggplot(
        data=(
            history.drop("worst", axis=1)
            .melt(id_vars="generation")
        ),
        mapping=pn.aes(x="generation", y="value", colour="variable"),
    )
    + pn.geom_line()
    + pn.theme(figure_size=[6, 3])
    # + pn.scale_y_log10()
)

NameError: name 'history' is not defined